In [1]:
# Evaluation on Testset

In [7]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as imgplot
import numpy as np
from sklearn.metrics import confusion_matrix
from keras.utils.vis_utils import model_to_dot

import h5py
import time
import tensorflow as tf
tf.set_random_seed(1)

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, BatchNormalization
from keras.layers import Convolution2D, MaxPooling2D, Flatten
import keras
import sys
print ("Keras {} TF {} Python {}".format(keras.__version__, tf.__version__, sys.version_info))

Keras 2.1.4 TF 1.2.1 Python sys.version_info(major=3, minor=6, micro=4, releaselevel='final', serial=0)


In [8]:
# Downloading the data, if it does not exist
import urllib
if not os.path.isfile('8_faces_test.hdf5'):
    urllib.request.urlretrieve(
    "https://www.dropbox.com/s/ugxrdo0lpc2ixvr/8_faces_test.hdf5?dl=1",
    "8_faces_test.hdf5")

In [12]:
import h5py

h5f_X = h5py.File('8_faces_train_and_valid.hdf5', 'r')
print(list(h5f_X.keys()))
X_train = h5f_X['X_train_8_faces']
print(X_train.shape)
Y_train_raw = h5f_X['Y_train_8_faces']
print(Y_train_raw.shape)
X_valid = h5f_X['X_valid_8_faces']
print(X_valid.shape)
Y_valid_raw = h5f_X['Y_valid_8_faces']
print(Y_valid_raw.shape)

class_labels = ["0 - Miranda Cosgrove", "1 - Chris Martin", "2 - Emma Stone", 
               "3 - Jamie Foxx", "4 - Steve Jobs", "5 - Zac Efron", "6 - Sandra Oh",
               "7 - Taryn Manning"]



['X_train_8_faces', 'X_valid_8_faces', 'Y_train_8_faces', 'Y_valid_8_faces']
(2000, 48, 48, 3)
(2000,)
(400, 48, 48, 3)
(400,)


In [13]:
h5f_X = h5py.File('8_faces_test.hdf5', 'r')
print(list(h5f_X.keys()))
X_test = h5f_X['X_test_8_faces']
print(X_test.shape)
Y_test = h5f_X['Y_test_8_faces']
print(Y_test.shape)

['X_test_8_faces', 'Y_test_8_faces']
(400, 48, 48, 3)
(400,)


In [14]:
X_mean = np.mean( X_train, axis = 0)
X_std = np.std( X_train, axis = 0)

X_train = (X_train - X_mean ) / (X_std + 0.0001)
X_valid = (X_valid - X_mean ) / (X_std + 0.0001)

In [15]:
X_test = (X_test - X_mean ) / (X_std + 0.0001)

In [19]:
model = Sequential()
name = '4CL_dropout_Datagen'

# Convolutional Part
model.add(Convolution2D(32, (3,3), input_shape=(48, 48, 3)))
#model.add(BatchNormalization())
model.add(Activation("relu"))

model.add(Convolution2D(32, (3,3)))
#model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D((2,2), 2))
model.add(Dropout(0.3))

model.add(Convolution2D(64, (3,3)))
#model.add(BatchNormalization())
model.add(Activation("relu"))

model.add(Convolution2D(64, (3,3)))
#model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D((2,2), 2))
model.add(Dropout(0.3))

model.add(Convolution2D(64, (3,3)))
#model.add(BatchNormalization())
model.add(Activation("relu"))

model.add(Convolution2D(64, (3,3)))
#model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D((2,2), 2))
model.add(Dropout(0.3))

# Dense Part
model.add(Flatten())

model.add(Dense(80))
model.add(Dropout(0.3))
model.add(Activation("relu"))

model.add(Dense(40))
model.add(Dropout(0.3))
model.add(Activation("relu"))
#model.add(BatchNormalization())

# Output Layer
model.add(Dense(8))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])
            

In [20]:
model.load_weights('Checkpoints/8_faces/4CL_dropout_Datagen/4CL_dropout_Datagenweights_epoch_050-0.98.hdf5')

In [21]:
preds_test=model.predict(X_test)

In [22]:
print(confusion_matrix(Y_test[:],np.argmax(preds_test,axis=1)))
print("Acc = " ,np.sum(Y_test[:]==np.argmax(preds_test,axis=1))/len(preds_test))

[[47  0  2  0  0  1  0  0]
 [ 0 21  2  5  5  7  9  1]
 [ 0  1 45  0  0  0  0  4]
 [ 0  0  2 47  0  0  1  0]
 [ 0  4  0  3 41  1  0  1]
 [ 6  0  1  0  0 43  0  0]
 [10  0  1  1  1  1 34  2]
 [ 6  3 13  1  0  2  2 23]]
Acc =  0.7525
